In [ ]:
#!/bin/bash
sudo apt update && sudo apt upgrade -y
sudo apt install software-properties-common -y
sudo add-apt-repository ppa:deadsnakes/ppa -y
sudo apt install python3.10 -y


In [ ]:
# cat hosts 
[amazon]

172.31.5.230  ansible_user="ec2-user" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"


[ubuntu]

172.31.2.1  ansible_user="ubuntu" ansible_port=22 ansible_ssh_private_key_file="ansible.pem

## wordpress-ubuntu-nginx

In [ ]:
# Basics need to know before installation 

sudo apt update 
sudo apt install mariadb-server 
sudo systemctl start mariadb
sudo systemctl enable mariadb


# cat /etc/mysql/my.cnf|grep -i sock
# Port or socket location where to connect
socket = /run/mysqld/mysqld.sock



In [ ]:
# vim nginx-php.vars
---
packages:
  - nginx
  - php8.1-fpm 
  - php8.1 
  - php8.1-common 
  - php8.1-mysql 
  - php8.1-xml 
  - php8.1-xmlrpc 
  - php8.1-curl 
  - php8.1-gd 
  - php8.1-imagick 
  - php8.1-cli 
  - php8.1-imap 
  - php8.1-mbstring 
  - php8.1-opcache 
  - php8.1-soap 
  - php8.1-zip 
  - php8.1-intl 
  - php8.1-bcmath 
  - unzip

# cat nginx-conf.vars 

nginx_port: "8080"
nginx_user: "www-data"
nginx_group: "www-data"
domain_name: "nginx.inenso.in"
wp_url: "https://wordpress.org/latest.tar.gz"

# cat mariadb-wordpress.vars 

maria_root_pwd: "mysqlroot@123*"
maria_user: "wordpressuser"
maria_db: "wordpresssdb"
maria_pwd: "wordpresspwd"
    
    

### mariadb installation, restart, root-password resetting,creation of wpdb,wpuser

In [ ]:
# cat ubuntu-nginx-wodpress.yml 
---
- name: "Installing LEMP on ubuntu"
  hosts: ubuntu
  become: yes
  vars_files:
    - nginx-conf.vars
    - mariadb-wordpress.vars
    - nginx-php.vars
  tasks:
    
    - name: " Installing Mariadb Packages and python3-pip"
      apt:
        name:
          - mariadb-server
          - python3-pip
        state: present
        update_cache: yes
    
    - name: "restart/enable Mariadb-server"
      service:
        name: mariadb
        state: restarted
        enabled: yes
    - name: "Python connector/module Installation"
      pip:
        name: PyMySQL
        state: present

    - name: "Resetting Mariadb root Password"
      ignore_errors: yes
      mysql_user:
        host_all: yes
        login_unix_socket: /run/mysqld/mysqld.sock
        login_password: ""
        login_user: root
        name: root 
        password: "{{ maria_root_pwd }}"

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP on ubuntu] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]

TASK [Installing Mariadb Packages and python3-pip] ********************************************************************************************************
ok: [172.31.12.224]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.12.224]

TASK [Python connector/module Installation] ***************************************************************************************************************
ok: [172.31.12.224]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.12.224]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.12.224]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.12.224]

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=7    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

In [ ]:
# On client machine

MariaDB [(none)]> select user,password,host from mysql.user;
+---------------+-------------------------------------------+-----------+
| User          | Password                                  | Host      |
+---------------+-------------------------------------------+-----------+
| mariadb.sys   |                                           | localhost |
| root          | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | localhost |
| mysql         | invalid                                   | localhost |
| wordpressuser | *0F882E7F510451CAAEA243E8217821FA7A4BB593 | %         |


MariaDB [(none)]> show grants for wordpressuser;
+--------------------------------------------------------------------------------------------------------------+
| Grants for wordpressuser@%                                                                                   |
+--------------------------------------------------------------------------------------------------------------+
| GRANT USAGE ON *.* TO `wordpressuser`@`%` IDENTIFIED BY PASSWORD '*0F882E7F510451CAAEA243E8217821FA7A4BB593' |
| GRANT ALL PRIVILEGES ON `wordpresssdb`.* TO `wordpressuser`@`%`                                              |
+--------------------------------------------------------------------------------------------------------------+
2 rows in set (0.000 sec

### nginx, php8.1-fpm and its extensions installation, restart/enable

In [ ]:
# cat ubuntu-nginx-wodpress.yml 
---
- name: "Installing LEMP on ubuntu"
  hosts: ubuntu
  become: yes
  vars_files:
    - nginx-conf.vars
    - mariadb-wordpress.vars
    - nginx-php.vars
  tasks:
    
    - name: " Installing Mariadb Packages and python3-pip"
      apt:
        name:
          - mariadb-server
          - python3-pip
        state: present
        update_cache: yes
    
    - name: "restart/enable Mariadb-server"
      service:
        name: mariadb
        state: restarted
        enabled: yes

    - name: "Python connector/module Installation"
      pip:
        name: PyMySQL
        state: present

    - name: "Resetting Mariadb root Password"
      ignore_errors: yes
      no_log: true
      mysql_user:
        host_all: yes
        login_unix_socket: /run/mysqld/mysqld.sock
        login_password: ""
        login_user: root
        name: root 
        password: "{{ maria_root_pwd }}"


    - name: "Creating Mariadb database for wordpress"
      mysql_db:
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_db }}"
        state: present


    
    - name: "Creating mariadb user for wp with privilege"
      no_log: true
      mysql_user:
        host: "%"
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_user }}"
        password: "{{ maria_pwd }}"
        priv: '{{ maria_db }}.*:ALL'
        state: present

    - name: "Nginx and php8.1-fpm and extensions Installation"
      apt:
        name: "{{ packages }}"
        state: present
        update_cache: yes
      
    - name: "Restart/enable for Nginx and php8.1-fpm Installation"
      service:
        enabled: yes
        name: "{{ item }}"
        state: restarted
      with_items:
        - nginx
        - php8.1-fpm


In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP on ubuntu] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]

TASK [Installing Mariadb Packages and python3-pip] ********************************************************************************************************
ok: [172.31.12.224]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.12.224]

TASK [Python connector/module Installation] ***************************************************************************************************************
ok: [172.31.12.224]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.12.224]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.12.224]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.12.224]

TASK [Nginx and php8.1-fpm and extensions Installation] ***************************************************************************************************
ok: [172.31.12.224]

TASK [Restart/enable for Nginx and php8.1-fpm Installation] ***********************************************************************************************
changed: [172.31.12.224] => (item=nginx)
changed: [172.31.12.224] => (item=php8.1-fpm)

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=9    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

### Creating document root, vhost, enabling vhost,restarting nginx,fpm

In [ ]:
# cat ubuntu-nginx-wodpress.yml 
---
- name: "Installing LEMP on ubuntu"
  hosts: ubuntu
  become: yes
  vars_files:
    - nginx-conf.vars
    - mariadb-wordpress.vars
    - nginx-php.vars
  tasks:
    
    - name: " Installing Mariadb Packages and python3-pip"
      apt:
        name:
          - mariadb-server
          - python3-pip
        state: present
        update_cache: yes
    
    - name: "restart/enable Mariadb-server"
      service:
        name: mariadb
        state: restarted
        enabled: yes

    - name: "Python connector/module Installation"
      pip:
        name: PyMySQL
        state: present

    - name: "Resetting Mariadb root Password"
      ignore_errors: yes
      no_log: true
      mysql_user:
        host_all: yes
        login_unix_socket: /run/mysqld/mysqld.sock
        login_password: ""
        login_user: root
        name: root 
        password: "{{ maria_root_pwd }}"


    - name: "Creating Mariadb database for wordpress"
      mysql_db:
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_db }}"
        state: present


    
    - name: "Creating mariadb user for wp with privilege"
      no_log: true
      mysql_user:
        host: "%"
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_user }}"
        password: "{{ maria_pwd }}"
        priv: '{{ maria_db }}.*:ALL'
        state: present

    - name: "Nginx and php8.1-fpm and extensions Installation"
      apt:
        name: "{{ packages }}"
        state: present
        update_cache: yes
      
    - name: "Restart/enable-nginx-php8.1-fpm"
      service:
        enabled: yes
        name: "{{ item }}"
        state: restarted
      with_items:
        - nginx
        - php8.1-fpm
      
    - name: "Creating Document root /var/www/{{ domain_name}}"
      file:
        group: "{{ nginx_group }}"
        owner: "{{ nginx_user }}"
        path: "/var/www/{{ domain_name }}"
        state: directory


    - name: "Creating nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
      template:
        dest: "/etc/nginx/sites-available/{{ domain_name }}.conf"
        src: vhost.tmpl

    - name: "Enabling nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
      file:
        src: "/etc/nginx/sites-available/{{ domain_name }}.conf"
        state: link
        path: "/etc/nginx/sites-enabled/{{ domain_name }}.conf"

    - name: "Restart/enable-nginx-php8.1-fpm"
      service:
        enabled: yes
        name: "{{ item }}"
        state: restarted
      with_items:
        - nginx
        - php8.1-fpm

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP on ubuntu] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]

TASK [Installing Mariadb Packages and python3-pip] ********************************************************************************************************
ok: [172.31.12.224]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.12.224]

TASK [Python connector/module Installation] ***************************************************************************************************************
ok: [172.31.12.224]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.12.224]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.12.224]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.12.224]

TASK [Nginx and php8.1-fpm and extensions Installation] ***************************************************************************************************
ok: [172.31.12.224]

TASK [Restart/enable-nginx-php8.1-fpm] ********************************************************************************************************************
changed: [172.31.12.224] => (item=nginx)
changed: [172.31.12.224] => (item=php8.1-fpm)

TASK [Creating Document root /var/www/nginx.inenso.in] ****************************************************************************************************
ok: [172.31.12.224]

TASK [Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.12.224]

TASK [Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.12.224]

TASK [Restart/enable-nginx-php8.1-fpm] ********************************************************************************************************************
changed: [172.31.12.224] => (item=nginx)
changed: [172.31.12.224] => (item=php8.1-fpm)

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=13   changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

### copying wordpress, wp-config.php and local files to document root 

In [ ]:
# cat ubuntu-nginx-wodpress.yml 
---
- name: "Installing LEMP with wordpress on ubuntu-22.04 LTS"
  hosts: ubuntu
  become: yes
  vars_files:
    - nginx-conf.vars
    - mariadb-wordpress.vars
    - nginx-php.vars
  tasks:
    
    - name: " Installing mariadb-server and python3-pip"
      apt:
        name:
          - mariadb-server
          - python3-pip
        state: present
        update_cache: yes
    
    - name: "restart/enable Mariadb-server"
      service:
        name: mariadb
        state: restarted
        enabled: yes

    - name: "Python connector/module Installation"
      pip:
        name: PyMySQL
        state: present

    - name: "Resetting Mariadb root Password"
      ignore_errors: yes
      no_log: true
      mysql_user:
        host_all: yes
        login_unix_socket: /run/mysqld/mysqld.sock
        login_password: ""
        login_user: root
        name: root 
        password: "{{ maria_root_pwd }}"


    - name: "Creating Mariadb database for wordpress"
      mysql_db:
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_db }}"
        state: present


    
    - name: "Creating mariadb user for wp with privilege"
      no_log: true
      mysql_user:
        host: "%"
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_user }}"
        password: "{{ maria_pwd }}"
        priv: '{{ maria_db }}.*:ALL'
        state: present

    - name: "Nginx and php8.1-fpm and extensions Installation"
      apt:
        name: "{{ packages }}"
        state: present
        update_cache: yes
      
    - name: "restart/enable-nginx-php8.1-fpm"
      service:
        enabled: yes
        name: "{{ item }}"
        state: restarted
      with_items:
        - nginx
        - php8.1-fpm
      
    - name: "Creating Document root /var/www/{{ domain_name}}"
      file:
        group: "{{ nginx_group }}"
        owner: "{{ nginx_user }}"
        path: "/var/www/{{ domain_name }}"
        state: directory


    - name: "Creating nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
      template:
        dest: "/etc/nginx/sites-available/{{ domain_name }}.conf"
        src: vhost.tmpl

    - name: "Enabling nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
      file:
        src: "/etc/nginx/sites-available/{{ domain_name }}.conf"
        state: link
        path: "/etc/nginx/sites-enabled/{{ domain_name }}.conf"

    - name: "restart/enable-nginx-php8.1-fpm"
      service:
        enabled: yes
        name: "{{ item }}"
        state: restarted
      with_items:
        - nginx
        - php8.1-fpm

    - name: "Copying info.php file"
      copy:
        dest: "/var/www/{{ domain_name }}"
        src: "{{ item }}"
        group: "{{ nginx_group }}"
        owner: "{{ nginx_user }}"
      with_items:
        - info.php
        - info.html
        
    - name: "Downloading wordpress"
      get_url:
        dest: /tmp/wordpress.tar.gz
        url: "{{ wp_url }}"

    - name: "Extracting wordpress files"
      unarchive:
        dest: /tmp/
        remote_src: yes
        src: /tmp/wordpress.tar.gz

    - name: "Copying wordpress files to document root /var/www/{{ domain_name }}"
      copy:
        remote_src: yes
        src: "/tmp/wordpress/"
        dest: "/var/www/{{ domain_name }}/"
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"

    - name: "copying/updating wp-config.php"
      template:
        dest: "/var/www/{{ domain_name }}/wp-config.php"
        src: wp-config-sample.php
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"

    - name: "cleanup"
      file:
        state: absent
        path: "{{ item }}"
      with_items:
        - /tmp/wordpress
        - /tmp/wordpress.tar.gz

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]

TASK [Installing mariadb-server and python3-pip] **********************************************************************************************************
ok: [172.31.12.224]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.12.224]

TASK [Python connector/module Installation] ***************************************************************************************************************
ok: [172.31.12.224]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.12.224]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.12.224]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.12.224]

TASK [Nginx and php8.1-fpm and extensions Installation] ***************************************************************************************************
ok: [172.31.12.224]

TASK [restart/enable-nginx-php8.1-fpm] ********************************************************************************************************************
changed: [172.31.12.224] => (item=nginx)
changed: [172.31.12.224] => (item=php8.1-fpm)

TASK [Creating Document root /var/www/nginx.inenso.in] ****************************************************************************************************
ok: [172.31.12.224]

TASK [Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.12.224]

TASK [Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.12.224]

TASK [restart/enable-nginx-php8.1-fpm] ********************************************************************************************************************
changed: [172.31.12.224] => (item=nginx)
changed: [172.31.12.224] => (item=php8.1-fpm)

TASK [Copying info.php file] ******************************************************************************************************************************
ok: [172.31.12.224] => (item=info.php)
ok: [172.31.12.224] => (item=info.html)

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [172.31.12.224]

TASK [Extracting wordpress files] *************************************************************************************************************************
changed: [172.31.12.224]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
ok: [172.31.12.224]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
ok: [172.31.12.224]

TASK [cleanup] ********************************************************************************************************************************************
changed: [172.31.12.224] => (item=/tmp/wordpress)
changed: [172.31.12.224] => (item=/tmp/wordpress.tar.gz)

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=19   changed=6    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

### --list-tasks --tags --step --start-at-task

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml --list-tasks

playbook: ubuntu-nginx-wodpress.yml

  play #1 (ubuntu): Installing LEMP with wordpress on ubuntu-22.04 LTS	TAGS: []
    tasks:
       Installing mariadb-server and python3-pip	TAGS: [lemp, mariadb]
      restart/enable Mariadb-server	TAGS: [lemp, mariadb]
      Python connector/module Installation	TAGS: [lemp, mariadb]
      Resetting Mariadb root Password	TAGS: [lemp, mariadb]
      Creating Mariadb database for wordpress	TAGS: [lemp, mariadb]
      Creating mariadb user for wp with privilege	TAGS: [lemp, mariadb]
      Installing Nginx and php8.1-fpm and extensions	TAGS: [lemp, nginx, php]
      restart/enable-nginx-php8.1-fpm	TAGS: [lemp, nginx]
      Creating Document root /var/www/{{ domain_name}}	TAGS: [lemp, nginx]
      Creating nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf	TAGS: [lemp, nginx]
      Enabling nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf	TAGS: [lemp, nginx]
      restart/enable-nginx-php8.1-fpm	TAGS: [lemp, nginx, php]
      Copying info.php file	TAGS: []
      Downloading wordpress	TAGS: [lemp, wp]
      Extracting wordpress files	TAGS: [lemp, wp]
      Copying wordpress files to document root /var/www/{{ domain_name }}	TAGS: [lemp, wp]
      copying/updating wp-config.php	TAGS: [lemp, nginx]
      cleanup	TAGS: [lemp, nginx]


In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml --tags=php

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]

TASK [Installing Nginx and php8.1-fpm and extensions] *****************************************************************************************************
ok: [172.31.12.224]

TASK [restart/enable-nginx-php8.1-fpm] ********************************************************************************************************************
changed: [172.31.12.224] => (item=nginx)
changed: [172.31.12.224] => (item=php8.1-fpm)

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=3    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml --start-at-task="Downloading wordpress"

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [172.31.12.224]

TASK [Extracting wordpress files] *************************************************************************************************************************
changed: [172.31.12.224]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
ok: [172.31.12.224]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
ok: [172.31.12.224]

TASK [cleanup] ********************************************************************************************************************************************
changed: [172.31.12.224] => (item=/tmp/wordpress)
changed: [172.31.12.224] => (item=/tmp/wordpress.tar.gz)

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=6    changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml --tags=mariadb --step

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************
Perform task: TASK: Gathering Facts (N)o/(y)es/(c)ontinue: yes

Perform task: TASK: Gathering Facts (N)o/(y)es/(c)ontinue: ************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.12.224]
Perform task: TASK:  Installing mariadb-server and python3-pip (N)o/(y)es/(c)ontinue: yes

Perform task: TASK:  Installing mariadb-server and python3-pip (N)o/(y)es/(c)ontinue: *********************************************************************

TASK [Installing mariadb-server and python3-pip] **********************************************************************************************************
ok: [172.31.12.224]
Perform task: TASK: restart/enable Mariadb-server (N)o/(y)es/(c)ontinue: yes

Perform task: TASK: restart/enable Mariadb-server (N)o/(y)es/(c)ontinue: **********************************************************************************

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.12.224]
Perform task: TASK: Python connector/module Installation (N)o/(y)es/(c)ontinue: N

Perform task: TASK: Python connector/module Installation (N)o/(y)es/(c)ontinue: ***************************************************************************
Perform task: TASK: Resetting Mariadb root Password (N)o/(y)es/(c)ontinue: c

Perform task: TASK: Resetting Mariadb root Password (N)o/(y)es/(c)ontinue: ********************************************************************************

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.12.224]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.12.224]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.12.224]

PLAY RECAP ************************************************************************************************************************************************
172.31.12.224              : ok=6    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

## Using ubuntu-nginx-wordpress-handler-tags

In [ ]:
# cat info.php 
<?php phpinfo(); ?>


In [ ]:
# cat wp-config-sample.php 
<?php
/**
 * The base configuration for WordPress
 *
 * The wp-config.php creation script uses this file during the installation.
 * You don't have to use the web site, you can copy this file to "wp-config.php"
 * and fill in the values.
 *
 * This file contains the following configurations:
 *
 * * Database settings
 * * Secret keys
 * * Database table prefix
 * * ABSPATH
 *
 * @link https://wordpress.org/support/article/editing-wp-config-php/
 *
 * @package WordPress
 */
define( 'WP_HOME', 'http://{{ domain_name }}' );
define( 'WP_SITEURL', 'http://{{ domain_name }}' );

// ** Database settings - You can get this info from your web host ** //
/** The name of the database for WordPress */
define( 'DB_NAME', '{{ maria_db }}' );

/** Database username */
define( 'DB_USER', '{{ maria_user }}' );

/** Database password */
define( 'DB_PASSWORD', '{{ maria_pwd }}' );

/** Database hostname */
define( 'DB_HOST', 'localhost' );


In [ ]:
# cat vhost.tmpl
server {
        listen {{ nginx_port }};
        listen [::]:{{ nginx_port }};
        root /var/www/{{ domain_name }};
        index index.php index.html index.htm;
        server_name {{ domain_name }};
 
        location / {
            try_files $uri $uri/ =404;
        }
 
        location ~ \.php$ {
            include snippets/fastcgi-php.conf;
            fastcgi_pass unix:/var/run/php/php8.1-fpm.sock;
        }
}


In [ ]:
# cat ubuntu-nginx-wodpress.yml 

---
- name: "Installing LEMP with wordpress on ubuntu-22.04 LTS"
  hosts: ubuntu
  become: yes
  vars_files:
    - nginx-conf.vars
    - mariadb-wordpress.vars
    - nginx-php.vars
  tasks:
    
    - name: " Installing mariadb-server and python3-pip"
      apt:
        name:
          - mariadb-server
          - python3-pip
        state: present
        update_cache: yes
      tags:
        - lemp 
        - mariadb
    
    - name: "restart/enable Mariadb-server"
      service:
        name: mariadb
        state: restarted
        enabled: yes
      tags:
        - lemp 
        - mariadb

    - name: "Python connector/module Installation"
      pip:
        name: PyMySQL
        state: present
      tags:
        - lemp 
        - mariadb    

    - name: "Resetting Mariadb root Password"
      ignore_errors: yes
      no_log: true
      mysql_user:
        host_all: yes
        login_unix_socket: /run/mysqld/mysqld.sock
        login_password: ""
        login_user: root
        name: root 
        password: "{{ maria_root_pwd }}"
      tags:
        - lemp 
        - mariadb


    - name: "Creating Mariadb database for wordpress"
      mysql_db:
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_db }}"
        state: present
      tags:
        - lemp 
        - mariadb

    
    - name: "Creating mariadb user for wp with privilege"
      no_log: true
      mysql_user:
        host: "%"
        login_password: "{{ maria_root_pwd }}"
        login_unix_socket: /run/mysqld/mysqld.sock
        login_user: root
        name: "{{ maria_user }}"
        password: "{{ maria_pwd }}"
        priv: '{{ maria_db }}.*:ALL'
        state: present
      tags:
        - lemp 
        - mariadb
    - name: "Installing Nginx and php8.1-fpm and extensions"
      apt:
        name: "{{ packages }}"
        state: present
        update_cache: yes
      tags:
        - lemp 
        - nginx
        - php
      notify:
        - restart/enable-nginx-php8.1-fpm
      
    - name: "Creating Document root /var/www/{{ domain_name }}"
      file:
        group: "{{ nginx_group }}"
        owner: "{{ nginx_user }}"
        path: "/var/www/{{ domain_name }}"
        state: directory
      tags:
        - lemp 
        - nginx


    - name: "Creating nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
      template:
        dest: "/etc/nginx/sites-available/{{ domain_name }}.conf"
        src: vhost.tmpl
      tags:
        - lemp 
        - nginx
      notify:
        - restart/enable-nginx-php8.1-fpm


    - name: "Enabling nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
      file:
        src: "/etc/nginx/sites-available/{{ domain_name }}.conf"
        state: link
        path: "/etc/nginx/sites-enabled/{{ domain_name }}.conf"
      tags:
        - lemp 
        - nginx
      notify:
        - restart/enable-nginx-php8.1-fpm

    - name: "Copying files to document root /var/www/{{ domain_name }}"
      copy:
        dest: "/var/www/{{ domain_name }}"
        src: "{{ item }}"
        group: "{{ nginx_group }}"
        owner: "{{ nginx_user }}"
      with_items:
        - info.php
        - info.html
        
    - name: "Downloading wordpress"
      get_url:
        dest: /tmp/wordpress.tar.gz
        url: "{{ wp_url }}"
      tags:
        - lemp 
        - wp

    - name: "Extracting wordpress files"
      unarchive:
        dest: /tmp/
        remote_src: yes
        src: /tmp/wordpress.tar.gz
      tags:
        - lemp 
        - wp 

    - name: "Copying wordpress files to document root /var/www/{{ domain_name }}"
      copy:
        remote_src: yes
        src: "/tmp/wordpress/"
        dest: "/var/www/{{ domain_name }}/"
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"
      tags:
        - lemp 
        - wp 

    - name: "copying/updating wp-config.php"
      template:
        dest: "/var/www/{{ domain_name }}/wp-config.php"
        src: wp-config-sample.php
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"
      tags:
        - lemp 
        - nginx

    - name: "cleanup"
      file:
        state: absent
        path: "{{ item }}"
      with_items:
        - /tmp/wordpress
        - /tmp/wordpress.tar.gz
      tags:
        - lemp 
        - nginx

  handlers:
    - name: "restart/enable-nginx-php8.1-fpm"
      service:
        name: "{{ item }}"
        enabled: yes
        state: restarted
      with_items:
        - nginx
        - php8.1-fpm 

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
The authenticity of host '172.31.2.1 (172.31.2.1)' can't be established.
ECDSA key fingerprint is SHA256:RXQhI1f04F704yigdMlyMnSTRxlRi1ocs7/joA3CGwk.
ECDSA key fingerprint is MD5:9e:66:6f:a8:4f:e4:00:63:f0:77:ef:6d:e8:06:4c:af.
Are you sure you want to continue connecting (yes/no)? yes
ok: [172.31.2.1]

TASK [Installing mariadb-server and python3-pip] **********************************************************************************************************
changed: [172.31.2.1]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.2.1]

TASK [Python connector/module Installation] ***************************************************************************************************************
changed: [172.31.2.1]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
changed: [172.31.2.1]

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
changed: [172.31.2.1]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
changed: [172.31.2.1]

TASK [Installing Nginx and php8.1-fpm and extensions] *****************************************************************************************************
changed: [172.31.2.1]

TASK [Creating Document root /var/www/nginx.inenso.in] ****************************************************************************************************
changed: [172.31.2.1]

TASK [Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
changed: [172.31.2.1]

TASK [Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
changed: [172.31.2.1]

TASK [Copying files to document root /var/www/nginx.inenso.in] ********************************************************************************************
changed: [172.31.2.1] => (item=info.php)
changed: [172.31.2.1] => (item=info.html)

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [172.31.2.1]

TASK [Extracting wordpress files] *************************************************************************************************************************
changed: [172.31.2.1]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
changed: [172.31.2.1]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
changed: [172.31.2.1]

TASK [cleanup] ********************************************************************************************************************************************
changed: [172.31.2.1] => (item=/tmp/wordpress)
changed: [172.31.2.1] => (item=/tmp/wordpress.tar.gz)

RUNNING HANDLER [restart/enable-nginx-php8.1-fpm] *********************************************************************************************************
changed: [172.31.2.1] => (item=nginx)
changed: [172.31.2.1] => (item=php8.1-fpm)

PLAY RECAP ************************************************************************************************************************************************
172.31.2.1                 : ok=18   changed=17   unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 

#### rerunning the same task

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.2.1]

TASK [Installing mariadb-server and python3-pip] **********************************************************************************************************
ok: [172.31.2.1]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.2.1]

TASK [Python connector/module Installation] ***************************************************************************************************************
ok: [172.31.2.1]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.2.1]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.2.1]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.2.1]

TASK [Installing Nginx and php8.1-fpm and extensions] *****************************************************************************************************
ok: [172.31.2.1]

TASK [Creating Document root /var/www/nginx.inenso.in] ****************************************************************************************************
ok: [172.31.2.1]

TASK [Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.2.1]

TASK [Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.2.1]

TASK [Copying files to document root /var/www/nginx.inenso.in] ********************************************************************************************
ok: [172.31.2.1] => (item=info.php)
ok: [172.31.2.1] => (item=info.html)

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [172.31.2.1]

TASK [Extracting wordpress files] *************************************************************************************************************************
changed: [172.31.2.1]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
ok: [172.31.2.1]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
ok: [172.31.2.1]

TASK [cleanup] ********************************************************************************************************************************************
changed: [172.31.2.1] => (item=/tmp/wordpress)
changed: [172.31.2.1] => (item=/tmp/wordpress.tar.gz)

PLAY RECAP ************************************************************************************************************************************************
172.31.2.1                 : ok=17   changed=4    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

#### nginx port chanaged

In [ ]:
# ansible-playbook -i hosts ubuntu-nginx-wodpress.yml 

PLAY [Installing LEMP with wordpress on ubuntu-22.04 LTS] *************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.2.1]

TASK [Installing mariadb-server and python3-pip] **********************************************************************************************************
ok: [172.31.2.1]

TASK [restart/enable Mariadb-server] **********************************************************************************************************************
changed: [172.31.2.1]

TASK [Python connector/module Installation] ***************************************************************************************************************
ok: [172.31.2.1]

TASK [Resetting Mariadb root Password] ********************************************************************************************************************
fatal: [172.31.2.1]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [Creating Mariadb database for wordpress] ************************************************************************************************************
ok: [172.31.2.1]

TASK [Creating mariadb user for wp with privilege] ********************************************************************************************************
ok: [172.31.2.1]

TASK [Installing Nginx and php8.1-fpm and extensions] *****************************************************************************************************
ok: [172.31.2.1]

TASK [Creating Document root /var/www/nginx.inenso.in] ****************************************************************************************************
ok: [172.31.2.1]

TASK [Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
changed: [172.31.2.1]

TASK [Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] ********************************************************************************
ok: [172.31.2.1]

TASK [Copying files to document root /var/www/nginx.inenso.in] ********************************************************************************************
ok: [172.31.2.1] => (item=info.php)
ok: [172.31.2.1] => (item=info.html)

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [172.31.2.1]

TASK [Extracting wordpress files] *************************************************************************************************************************
changed: [172.31.2.1]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
ok: [172.31.2.1]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
ok: [172.31.2.1]

TASK [cleanup] ********************************************************************************************************************************************
changed: [172.31.2.1] => (item=/tmp/wordpress)
changed: [172.31.2.1] => (item=/tmp/wordpress.tar.gz)

RUNNING HANDLER [restart/enable-nginx-php8.1-fpm] *********************************************************************************************************
changed: [172.31.2.1] => (item=nginx)
changed: [172.31.2.1] => (item=php8.1-fpm)

PLAY RECAP ************************************************************************************************************************************************
172.31.2.1                 : ok=18   changed=6    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1